In [37]:
import json
source1 =  {"items":[{"id":"ALZ_SE_SIM_CUSTOM_TEMPLATE://eing_abs/AJ", "facts": [{"attributeId":"core.classType","value":"com.infa.ldm.relational.Column"}, {"attributeId":"core.name","value":"AJ"}]}, {"id":"ALZ_SE_SIM_CUSTOM_TEMPLATE://eing_abs/FS_SEGMENT", "facts": [{"attributeId":"core.classType","value":"com.infa.ldm.relational.Column"}, {"attributeId":"core.name","value":"FS_SEGMENT"}]}],"metadata":{"totalCount":93}}
df = sqlContext.read.json(sc.parallelize([json.dumps(source1)]),multiLine=True)
df.show(truncate=False)
df.printSchema()

StatementMeta(sparkPool1, 13, 10, Finished, Available)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|items                                                                                                                                                                                                                                             |metadata|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|[{[{core.classType, com.infa.ldm.relational.Column}, {core.name, AJ}], ALZ_SE_SIM_CUSTOM_TEMPLATE://eing_abs/AJ}, {[{core.classType, com.infa.ldm.relational.Column}, {core.name, FS_SEGMENT}], ALZ_SE_SIM_CUSTOM_TEMPLATE://eing_abs/FS_SEGM

In [45]:
from pyspark.sql.functions import explode, arrays_zip, col, expr
df1 = (df.withColumn('buffer', explode('items'))
         .withColumn('facts',expr("buffer.facts"))
         .withColumn('id',expr("buffer.id"))
         .drop(*['items','metadata','buffer'])
)
df1.show(truncate=False)

StatementMeta(sparkPool1, 13, 18, Finished, Available)

+---------------------------------------------------------------------------+------------------------------------------------+
|facts                                                                      |id                                              |
+---------------------------------------------------------------------------+------------------------------------------------+
|[{core.classType, com.infa.ldm.relational.Column}, {core.name, AJ}]        |ALZ_SE_SIM_CUSTOM_TEMPLATE://eing_abs/AJ        |
|[{core.classType, com.infa.ldm.relational.Column}, {core.name, FS_SEGMENT}]|ALZ_SE_SIM_CUSTOM_TEMPLATE://eing_abs/FS_SEGMENT|
+---------------------------------------------------------------------------+------------------------------------------------+



In [47]:
from pyspark.sql import functions as F
df1 = df1.withColumn(
    "header", 
    F.expr("filter(facts, x -> x.attributeId = 'core.name')")[0]["value"]
)
df1.show()

StatementMeta(sparkPool1, 13, 20, Finished, Available)

+--------------------+--------------------+----------+
|               facts|                  id|    header|
+--------------------+--------------------+----------+
|[{core.classType,...|ALZ_SE_SIM_CUSTOM...|        AJ|
|[{core.classType,...|ALZ_SE_SIM_CUSTOM...|FS_SEGMENT|
+--------------------+--------------------+----------+

